In [6]:
#%pylab
#%matplotlib inline
#%load_ext watermark
#%watermark -v -p numpy,tensorflow

MNIST 구현을 위해 다음 두 개 라이브러리 임포트가 필요합니다. 

In [6]:
import tensorflow as tf
import numpy as np

MNIST의 데이터는 텐서플로에서 제공합니다. 그래서 따로 다운로드 할 필요 없이 임포트해 사용하면 됩니다.

In [ ]:
# MNIST_data download(traing:55000, test:10000)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

하지만 텐서플로 1.7.0 버전에서부터는 샘플 데이터를 다운로드 하는 기능이 제외될 예정이라는 경고가 발생합니다. 대신 케라스(Keras)를 사용하여 MNIST 데이터를 다운받기를 권장해 드립니다.

In [4]:
# MNIST_data download(traing:60000, test:10000)
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

데이터 셋의 구성을 확인해보겠습니다. training 데이터는 (60000, 784) 되어 있습니다. 첫 번째 차원은 각 이미지에 대한 인덱스이며 두 번째 차원은 이미지 안의 픽셀 수를 나타냅니다. 이미지 크기가 가로 28, 세로 28픽셀로 되어있어 28*28=784 가 됩니다.
정답 데이터는 one-hot encoding 방식으로 예를 들어 2이미지의 벡터는[0, 1, 0, …., 0]입니다.

In [5]:
print(tf.convert_to_tensor(X_train).get_shape())
print(tf.convert_to_tensor(X_test).get_shape())
print(tf.convert_to_tensor(y_train).get_shape())
print(tf.convert_to_tensor(y_test).get_shape())

(60000, 784)
(10000, 784)
(60000, 10)
(10000, 10)


배치 데이터를 만들기 위해 파이썬 제너레이터 함수를 정의합니다.

In [10]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

가중치 텐서와 바이어스 텐서를 만듭니다. tf.Variable 함수를 사용하여 생성되었고 초기 값을 가집니다. 여기서는 모두 초기 값을 0으로 이루어진 상수 텐서를 생성하였습니다.

In [8]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
print(W.get_shape())
print(b.get_shape())

(784, 10)
(10,)


훈련 이미지 데이터를 넣을 플레이스홀더를 2차원 텐서로 만듭니다. 텐서 x는 MNIST 이미지를 784픽셀을 실수로 저장하는 벡터로 사용됩니다. None이라고 지정한 것은 어떤 크기나 가능하다는 뜻으로서, 여기에서는 학습 과정에 사용될 이미지의 총 개수가 될 것입니다.

In [12]:
x = tf.placeholder("float", [None, 784])

텐서를 만들었으니 모델을 만들 차례입니다. tf.nn.softmax() 함수는 소프트맥스 함수를 텐서플로로 구현한 것입니다.

In [13]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

비용함수는 교차 엔트로피를 이용하겠습니다. 그리고 가중치(W) 및 편향(b) 조정을 위해 역전파를 사용하였습니다.
참고로 비용함수는 여러가지가 있습니다.(평균제곱오차, 유클리드 제곱거리, 교차 엔트로피)
비용함수 내용은 https://ynebula.tistory.com/28 참고바랍니다.

비용함수는 교차 엔트로피를 이용하였습니다. 그리고 가중치(W) 및 편향(b) 조정을 위해 역전파를 사용하였습니다. 교차 엔트로피 비용함수와 경사하강법을 사용하면, 매 루프마다 오차를 줄이기 위해서는 주어진 상황에서 얼마만큼 매개변수를 변경해야할지를 계산할 수 있습니다. 
비용함수 교차 엔트로피를 구현하기 위해서는 실제 레이블을 담고 있는 새로운 플레이스홀더가 하나 더 필요합니다. 이 플레시스홀더를 이용해 다음 코드처럼 크로스 엔트로피 비용함수를 구현할 수 있습니다. 먼저 텐서플로 내장 함수인 tf.log()를 사용해 y의 각 원소 로그 값을 구합니다. 그러고 나서 y_의 각 원소와 곱합니다. 마지막으로 tf.reduce_sum() 함수를 사용하여 텐서의 모든 원소를 더합니다.
나중에 다룰 이미지는 여러 개이므로 교차 엔트로피의 값은 하나의 이미지에 대한 것이 아니라 이미지 묶음 y에 대응하여 나옵니다.

참고로 비용함수는 여러 가지 있습니다(평균제곱오차, 유클리드 제곱거리, 교차 엔트로피). 비용함수는  https://ynebula.tistory.com/28 를 참고바랍니다.

실제 레이블을 담기 위한 텐서와 교차 엔트로피 방식을 이용하는 그래디언트디센트 방식을 선택합니다.

In [14]:
y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

학습률은 0.01로 설정하겠습니다. 학습률은 tf.train.GradientDescentOptimizer() 매개변수로 전달합니다. 경사하강법 함수는 미분을 자동으로 연산해 줍니다. 즉, 학습 과정에서 가중치와 편향 조정을 위한 필요한 미분 연산을 텐서플로가 자동으로 처리합니다.
minimize() 메서드가 실행될 때 텐서플로 손실함수(loss function), 즉 비용함수에 연관된 변수들을 알아서 인식하고 각각에 대해 기울기를 계산합니다.
텐서플로 사용자는 예측 모델의 계산 구조를 정의하고 목표함수(y=W*x+b)와 연결한 후 데이터만 넣어주면 됩니다.

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

알고리즘을 모두 작성했으니, tf.Session()으로 시스템에서 사용 가능한 디바이스(CPU or GPU)에서 텐서플로의 연산을 실행합니다. 모든 변수를 초기화하고 세션을 시작합니다..

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

여기서부터 모델 훈련을 시작합니다. train_step에서 산출된 매개변수는 경사하강법 알고리즘에 다시 참여하게 됩니다. 따라서 모델을 훈련시키려면 train_step을 반복적으로 실행해야 합니다. 한번 train_step을 1000번 실행해보겠습니다. 루프 내 첫 번째 라인은 훈련 데이터 셋으로부터 무작위로 100개 추출합니다. 루프를 반복할 때마다 전체 데이터를 모두 사용할 수도 있지만, 예제를 더 편리하게 실습하기 위해 작은 표본 데이터를 이용하는 방법을 택했습니다. 그 다음 라인에서는 플레이스홀더를 사용하여 이 100개의 샘플 데이터를 주입니다. 

생성된 학습 모델이 좋은 모델인지 판단하기 위해서 모델평가를 수행해야 합니다. tf.argmax(y,1)은 입력 이미지에 대해 가장 높은 확률을 가진 레이블을 리턴 합니다. tf.argmax(y_,1)은 실제 레이블 입니다. tf.equal() 함수를 사용하여 예측 값과 실제 레이블을 다음과 같이 비교할 수 있습니다.

위 코드는 불리언으로 이루어진 리스트를 리턴 합니다. 예측한 것이 얼만큼 맞았는지를 확인하려면 불리언을 수치 값으로 다음과 같이 변경합니다. 예를 들어 [True, False, True, True]는 [1, 0, 1, 1]로 바뀔 테고, 그 평균은 0.75가 나올 것입니다. 이것이 바로 정확도(accuracy)의 퍼센트를 나타냅니다. 이제 feed_dict 매개변수로 mnist.test를 전달하여 테스트 데이터 셋에 대한 정확도를 계산해봅시다. 1000의 반복을 수행하고 결과를 출력합니다.

In [ ]:
for i in range(1000):
    batch_xs, batch_ys = next(shuffle_batch(X_train, y_train, 100))
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    if i % 100 == 0:
        print(sess.run(accuracy, feed_dict={x: X_test, y_: y_test}))

0.2756
0.8862
0.9004
0.908
0.9115
0.9086
0.9126
0.9161
0.9131
0.9127


In [19]:
x = tf.constant([[1, 1, 1], [1, 1, 1]])
print(tf.reduce_sum(x))  # 6
print(tf.reduce_sum(x, 0))  # [2, 2, 2]
print(tf.reduce_sum(x, 1))  # [3, 3]
print(tf.reduce_sum(x, 1, keepdims=True))  # [[3], [3]]
print(tf.reduce_sum(x, [0, 1]))  # 6

Tensor("Sum_6:0", shape=(), dtype=int32)
Tensor("Sum_7:0", shape=(3,), dtype=int32)
Tensor("Sum_8:0", shape=(2,), dtype=int32)
Tensor("Sum_9:0", shape=(2, 1), dtype=int32)
Tensor("Sum_10:0", shape=(), dtype=int32)
